In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import matplotlib.font_manager as fm
import matplotlib as mpl
from matplotlib import rc
from scipy.stats import mannwhitneyu
import datetime
import json
import pickle
from tqdm import tqdm
import warnings
import lightgbm as lgb
import math
import random
import re
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from itertools import permutations

warnings.filterwarnings(action='ignore')


from sklearn.metrics import *
from sklearn.model_selection import KFold

def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)

def split_time(x):
    x = str(x)
    year = x[:4]
    month = x[4:6]
    day = x[6:8]
    hour = x[8:10]
    minute = x[10:12]
    second = x[12:14]
    return year, month, day, hour, minute, second

def make_time_cols(df_):
    years, months, days, hours, minutes, seconds, dates = [], [], [], [], [], [], []

    for i in range(len(df_)):
        x = df_.loc[i,'time']
        year, month, day, hour, minute, second = split_time(x)
        years.append(int(year))
        months.append(int(month))
        days.append(int(day))
        hours.append(int(hour))
        minutes.append(int(minute))
        seconds.append(int(second))
        dates.append(datetime.datetime.strptime(str(x), "%Y%m%d%H%M%S"))

    df_['year'] = years
    df_['month'] = months
    df_['day'] = days
    df_['hour']=hours
    df_['minute'] = minutes
    df_['second']= seconds
    df_['date'] = dates
    return df_


def add_time_cols(df_):
    df_ = make_time_cols(df_)
    df_['weeknum'] = df_['date'].apply(lambda x: x.weekday())
    df_['weekend'] = df_['weeknum'].apply(lambda x: 1 if x >=5 else 0)
    df_['md'] = df_['month'].apply(lambda x: str(x).zfill(2)) + df_['day'].apply(lambda x: str(x).zfill(2))
    return df_

In [2]:
default_path = os.getcwd()
data_path = os.path.join(default_path,'data')

In [3]:
train_err = pd.read_csv(data_path+'/train_err_data_c.csv', parse_dates=['date'])
#test_err = pd.read_csv(data_path+'/test_err_data_c.csv')

In [4]:
train_prob = pd.read_csv(data_path+'/train_problem_data_c.csv', parse_dates=['date'])
prob_user_ids = np.unique(train_prob['user_id'])
err_user_ids = np.unique(train_err['user_id'])
y_users = prob_user_ids.tolist()
n_users = np.setdiff1d(err_user_ids, prob_user_ids).tolist()\

## EDA로 얻은 insight

#### 1. model_mn과 fwver은 1:m 대응관계임
* model_mn의 version은 fwver임
  * model_mn 1개당 여러 fwver을 가질 수 있음
  * fwver의 값이 작아지는 경우 downgrade로 가정함
    * downgrade 하는 경우 93개 중 63개가 불만 접수 함
    
* version 변화 : model_1 -> model_2로 변화할 때 특히 불만 접수가 커짐
  * (04.16.3553) 기준
    * 04.16.3553 -> 04.33.1261인 경우 전체 599개 중 569개가 불만 접수 됨 (model_1 -> model_2)
    * 04.16.3553 -> 04.33.1185인 경우 전체 161개 중 151개가 불만 접수 됨 (model_1 -> model_2)
    * 04.16.3553 -> 04.33.1149인 경우 전체 86개 중 85개가 불만 접수 됨 (model_1 -> model_2)
    * 04.16.3553 -> 04.16.3571인 경우 전체 499개 중 293개가 불만 접수 됨 (model_1 -> model_1)
    * 04.16.3439 -> 04.16.3553인 경우 전체 5개 중 1개가 불만 접수 됨 (model_1 -> model_1)

  * (04.22.1750) 기준
    * 04.22.1750 -> 04.22.1778인 경우 전체 4133개 중 1568개가 불만 접수 됨 (model_0 -> model_0)
    * 04.22.1684 -> 04.22.1750인 경우 전체 32개 중 17개가 불만 접수 됨 (model_0 -> model_0)
    * 04.22.1684 -> 04.22.1778인 경우 전체 19개 중 10개가 불만 접수 됨 (model_0 -> model_0)
    * 04.22.1750 -> 04.22.1684인 경우 전체 6개 중 6개가 불만 접수 됨 (model_0 -> model_0)

  * 04.33.1185 -> 04.33.1261인 경우 전체 3185개 중 946개가 불만 접수 됨 (model_2 -> model_2)

In [10]:
def make_change_dict(err_df, data_path, option = 'train'):
    change_dict = {}
    change_dict['id'] = []
    change_dict['upgrade'] = []
    change_dict['downgrade'] = []
    change_dict['change_model'] = []
    change_dict['change_version'] = []
    change_dict['change_all'] = []
    all_ids = np.unique(err_df['user_id'])
    
    if not os.path.isfile(f"{data_path}/{option}_upgrade_dict.json"):
        for id_ in tqdm(all_ids):
            temp_df = err_df[err_df['user_id']==id_].sort_values('date').reset_index(drop=True)

            unique_model = np.unique(temp_df['model_nm'])
            unique_version = np.unique(temp_df['fwver'])

            if len(unique_model) != 1 or len(unique_version)!=1:

                past_version = temp_df.loc[0,'fwver']
                past_model = temp_df.loc[0,'model_nm']
                upgrade = 0
                downgrade = 0
                change_model = 0
                change_version = 0
                change_all = 0

                for i in range(1,len(temp_df)):
                    cur_version = temp_df.loc[i,'fwver']
                    cur_model = temp_df.loc[i,'model_nm']
                    sw = 0
                    if past_model != cur_model:
                        change_model +=1
                        change_all +=1
                        sw = 1
                        
                    if past_version != cur_version:
                        change_version +=1
                        
                        past_version_f = past_version.split('.')
                        past_version_f = float(past_version_f[0] + '.' + ''.join(past_version_f[1:]))

                        cur_version_f = cur_version.split('.')
                        cur_version_f = float(cur_version_f[0] + '.' + ''.join(cur_version_f[1:]))
                        if past_model != cur_model and past_version_f > cur_version_f:
                            downgrade +=1
                        elif past_model != cur_model and past_version_f < cur_version_f:
                            upgrade +=1
                        if sw == 0:
                            change_all +=1  
                        
                    past_version = cur_version
                    past_model = cur_model

                change_dict['id'].append(id_)
                change_dict['upgrade'].append(upgrade)
                change_dict['downgrade'].append(downgrade)
                change_dict['change_model'].append(change_model)
                change_dict['change_version'].append(change_version)
                change_dict['change_all'].append(change_all)

        change_dict['id'] =list(map(int, change_dict['id']))
        with open(f"{data_path}/{option}_upgrade_dict.json", "w") as f:
            json.dump(change_dict, f)
    else:
        pass
    
    
def extract_change_df(change_dict):

    change_df = pd.DataFrame({'user_id':change_dict['id'], 'upgrade':change_dict['upgrade'],
                         'downgrade':change_dict['downgrade'], 'change_model':change_dict['change_model'],
                         'change_version':change_dict['change_version'],
                         'change_all':change_dict['change_all']})
    return change_df

In [6]:
temp_df = train_err.groupby(['model_nm','fwver']).sum().reset_index()
t_temp_df = temp_df.groupby('fwver').size().to_frame('size').reset_index()
t_temp_df

,fwver,size
0,03.11.1141,1
1,03.11.1149,1
2,03.11.1167,1
3,04.16.2641,1
4,04.16.3345,1
5,04.16.3439,1
6,04.16.3553,1
7,04.16.3569,1
8,04.16.3571,1
9,04.22.1442,1


In [11]:
if not os.path.isfile(f"{data_path}/train_upgrade_dict.json"):
    make_change_dict(train_err, data_path, option = 'train')
if not os.path.isfile(f"{data_path}/test_upgrade_dict.json"):
    make_change_dict(test_err, data_path, option = 'test')
    
with open(f"{data_path}/train_upgrade_dict.json", "r") as f:
    train_change_dict = json.load(f)
with open(f"{data_path}/test_upgrade_dict.json", "r") as f:
    test_change_dict = json.load(f)

In [12]:
train_change_df = extract_change_df(train_change_dict)
test_change_df = extract_change_df(test_change_dict)

In [17]:
temp_df = train_change_df[(train_change_df['downgrade']>0)]
len(temp_df['user_id'])

93

In [18]:
len(np.intersect1d(temp_df['user_id'], prob_user_ids))

63

#### 2. 일간 에러 수가 급격히 감소하는 경우 불만 제기 이후 결과일 수 있음
* 패턴 관측을 위해 11월 1일 ~ 30일까지 각 user의 일간 error 개수를 변수로 함(30개 변수)
* error가 없는 이유는 기계가 작동하지 않아서일 수도 있다고 추정

In [29]:
train_prob.loc[2,'date']

Timestamp('2020-11-14 13:00:00')

In [32]:
train_err[train_err['user_id']==prob_user_ids[2]][['date','errtype','errcode']].iloc[200:250]

,date,errtype,errcode
4271,2020-11-11 18:07:28,31,1
4272,2020-11-11 18:07:39,31,0
4273,2020-11-11 19:32:20,24,1
4274,2020-11-11 19:35:26,23,connection timeout
4275,2020-11-11 19:35:27,22,1
4276,2020-11-11 22:02:17,15,1
4277,2020-11-12 11:33:49,31,1
4278,2020-11-12 11:33:49,26,1
4279,2020-11-12 11:33:49,16,1
4280,2020-11-12 11:34:01,31,0


#### 3. 각 user의 errtype 개수
err별 비교
* 등장 자체가 영향?
  * errtype 18 : 1768개 중 1521개인 80% 이상이 불만제기
  * errtype 19 : 419개 중 346개인 80% 이상이 불만제기
  * errtype 20 : 1546개 중 1333개인 80% 이상이 불만제기
  * errtype 21 : 381개 중 316개인 80% 이상이 불만제기
  * errtype 24 
    * connection fail for LMP response timout : 114개 중 62%가 불만제기
    * terminate by peer user : 1225개 중 74%가 불만제기
  * errtype 25 : 420개 중 360개인 80% 이상이 문제 제기
    * 전체 errcode가 80% 넘어감
  * errtype 30 : 420개 중 360개인 69% 이상이 문제 제기
    * 전체 errcode가 60% 넘어감
  * errtype 34
    * 2, 3 : 60% 넘어감
  * errtype 37
    * 1 : 68% 넘어감

#### 4. 각 user의 총 error 개수
* 각 user에서 error 개수가 많을 수록 불만 접수 확률이 높아질 것이라 추정하여 변수 넣음

#### 5. quality
* 검사횟수, 평균 quality 값이 불만 접수에 영향을 줄 것이라 판단하여 변수 넣음
  * 보면 같은 user를 같은 시간에 12, 24번 검사하는데 여기서 quality 결과값이 다른 경우가 있음
    * 표준편차 값도 반영 예정

#### 6. errtype + errcode
* train과 test에서 겹치는 경우가 96개밖에 없음
  * errtype과 errcode를 합쳐 쓰기는 어려울 것으로 보임
* train 기준, errtype과 errcode의 조합은 다음과 같음
  * errtype ; errtype 번호, len : errtype에 있는 errcode의 개수

In [33]:
"""
errtype : 1, len : 8
['0' 'P-41001' 'P-41007' 'P-41007 ' 'P-41011' 'P-41011 ' 'P-44010'
 'P-44010 ']
errtype : 2, len : 2
['0' '1']
errtype : 3, len : 3
['0' '1' '2']
errtype : 4, len : 2
['0' '1']
errtype : 5, len : 65
['0001' '2638' '40013' '60045' '700001' 'B-51042' 'B-51049' 'B-A8002'
 'C-11017' 'C-11020' 'C-11087' 'C-12032' 'C-13039' 'C-13053' 'C-14014'
 'CM a' 'D-10011' 'D-99999' 'E-59902' 'En00402' 'En00406' 'En00409'
 'H-51042' 'H-51046' 'H-51048' 'H-51049' 'J-20029' 'J-30010' 'J-30021'
 'J-40011' 'M-51007' 'M-51020' 'M-99999' 'P_41001' 'P_41007' 'Q-64001'
 'Q-64002' 'Q-73004' 'Q-73006' 'S-61001' 'S-64000' 'S-64001' 'S-64002'
 'S-65' 'S-65002' 'U-81000' 'U-81009' 'U-81014' 'U-82004' 'U-82020'
 'U-82023' 'U-82024' 'U-82026' 'V-21002' 'V-21003' 'V-21004' 'V-21005'
 'V-21007' 'V-21008' 'V-21010' 'Y-00004' 'Y-00005' 'Y-00008' 'http' 'nan']
errtype : 6, len : 2
['1' '14']
errtype : 7, len : 2
['1' '14']
errtype : 8, len : 3
['20' 'PHONE_ERR' 'PUBLIC_ERR']
errtype : 9, len : 9
['1' 'C-11020' 'C-12032' 'C-13039' 'C-14014' 'V-21002' 'V-21004' 'V-21005'
 'V-21008']
errtype : 10, len : 1
['1']
errtype : 11, len : 1
['1']
errtype : 12, len : 1
['1']
errtype : 13, len : 1
['1']
errtype : 14, len : 3
['1' '13' '14']
errtype : 15, len : 1
['1']
errtype : 16, len : 1
['1']
errtype : 17, len : 5
['1' '12' '13' '14' '21']
errtype : 18, len : 1
['1']
errtype : 19, len : 1
['1']
errtype : 20, len : 1
['1']
errtype : 21, len : 1
['1']
errtype : 22, len : 1
['1']
errtype : 23, len : 8
['UNKNOWN' 'active' 'connection fail for LMP response timout'
 'connection fail to establish' 'connection timeout'
 'connectionterminated by local host' 'standby' 'terminate by peer user']
errtype : 24, len : 1
['1']
errtype : 25, len : 9
['1' '2' 'L2CAP connection cancelled' 'UNKNOWN'
 'connection fail to establish' 'connection timeout'
 'connectionterminated by local host' 'scanning timeout'
 'terminate by peer user']
errtype : 26, len : 1
['1']
errtype : 27, len : 1
['1']
errtype : 28, len : 1
['1']
errtype : 30, len : 5
['0' '1' '2' '3' '4']
errtype : 31, len : 2
['0' '1']
errtype : 32, len : 55
['-269' '-270' '100' '101' '102' '103' '104' '105' '106' '107' '108' '109'
 '110' '37' '38' '41' '42' '46' '47' '62' '65' '66' '67' '68' '69' '70'
 '71' '72' '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84'
 '85' '86' '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98'
 '99']
errtype : 33, len : 3
['1' '2' '3']
errtype : 34, len : 6
['1' '2' '3' '4' '5' '6']
errtype : 35, len : 1
['1']
errtype : 36, len : 1
['8.0']
errtype : 37, len : 2
['0' '1']
errtype : 38, len : 2653
['10005' '10018' '10043' ... '9971' '9973' '9983']
errtype : 39, len : 2
['0' '1']
errtype : 40, len : 2
['0' '1']
errtype : 41, len : 1
['NFANDROID2']
errtype : 42, len : 2
['2' '3']
"""

"\nerrtype : 1, len : 8\n['0' 'P-41001' 'P-41007' 'P-41007 ' 'P-41011' 'P-41011 ' 'P-44010'\n 'P-44010 ']\nerrtype : 2, len : 2\n['0' '1']\nerrtype : 3, len : 3\n['0' '1' '2']\nerrtype : 4, len : 2\n['0' '1']\nerrtype : 5, len : 65\n['0001' '2638' '40013' '60045' '700001' 'B-51042' 'B-51049' 'B-A8002'\n 'C-11017' 'C-11020' 'C-11087' 'C-12032' 'C-13039' 'C-13053' 'C-14014'\n 'CM a' 'D-10011' 'D-99999' 'E-59902' 'En00402' 'En00406' 'En00409'\n 'H-51042' 'H-51046' 'H-51048' 'H-51049' 'J-20029' 'J-30010' 'J-30021'\n 'J-40011' 'M-51007' 'M-51020' 'M-99999' 'P_41001' 'P_41007' 'Q-64001'\n 'Q-64002' 'Q-73004' 'Q-73006' 'S-61001' 'S-64000' 'S-64001' 'S-64002'\n 'S-65' 'S-65002' 'U-81000' 'U-81009' 'U-81014' 'U-82004' 'U-82020'\n 'U-82023' 'U-82024' 'U-82026' 'V-21002' 'V-21003' 'V-21004' 'V-21005'\n 'V-21007' 'V-21008' 'V-21010' 'Y-00004' 'Y-00005' 'Y-00008' 'http' 'nan']\nerrtype : 6, len : 2\n['1' '14']\nerrtype : 7, len : 2\n['1' '14']\nerrtype : 8, len : 3\n['20' 'PHONE_ERR' 'PUBLIC_ERR']\